In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

def load_data(file_path):
    return pd.read_csv(file_path)

def preprocess_data(women_data, population_data):
    women_data['STATE/UT'] = women_data['STATE/UT'].str.upper()
    statewise = women_data.groupby(['STATE/UT', 'Year']).sum().reset_index()
    year_wise = statewise.groupby('Year')[['Rape', 
    'Kidnapping and Abduction', 
    'Dowry Deaths', 
    'Assault on women with intent to outrage her modesty', 
    'Insult to modesty of Women', 
    'Cruelty by Husband or his Relatives', 
    'Importation of Girls']].sum()
    year_wise['Total'] = year_wise[['Rape', 
    'Kidnapping and Abduction', 
    'Dowry Deaths', 
    'Assault on women with intent to outrage her modesty', 
    'Insult to modesty of Women', 
    'Cruelty by Husband or his Relatives', 
    'Importation of Girls']].sum(axis=1)
    merged_df = pd.merge(year_wise, population_data[['Year', 'Population', 'Literacy Rate']], on='Year', how='inner')
    merged_df['Crime_Rate'] = merged_df['Total'] / merged_df['Population'] * 1_000_000
    return merged_df

In [16]:

# Load data
women_data = load_data('../Data/women/combined_data.csv')
population_data = load_data('../population.csv')

# Preprocess data
merged_df = preprocess_data(women_data, population_data)
merged_df


,Year,Rape,Kidnapping and Abduction,Dowry Deaths,Assault on women with intent to outrage her modesty,Insult to modesty of Women,Cruelty by Husband or his Relatives,Importation of Girls,Total,Population,Literacy Rate,Crime_Rate
0,2001,32150,29290,13702,68248,19492,98340,228,261450,1078970907,64.83,242.314226
1,2002,32746,29012,13644,67886,20310,98474,152,262224,1098313039,65.69,238.751604
2,2003,31694,26592,12416,65878,24650,101406,92,262728,1117415123,66.57,235.121214
3,2004,36466,31156,14052,69134,20002,116242,178,287230,1136264583,67.46,252.784434
4,2005,36718,31500,13574,68350,19968,116638,298,287046,1154638713,68.36,248.602439
5,2006,38696,34828,15236,73234,19932,126256,134,308316,1172373788,69.28,262.984385
6,2007,41474,40832,16186,77468,21900,151860,122,349842,1189691809,70.20,294.061031
7,2008,42934,45878,16344,80826,24428,162688,134,373232,1206734806,71.14,309.290822
8,2009,42794,51482,16766,77422,22018,179092,96,389670,1223640160,72.09,318.451464
9,2010,44344,59590,16782,81226,19922,188082,72,410018,1240613620,73.06,330.496130
